# TP2
### Problema 2

O Conway’s Game of Life é um exemplo bastante conhecido de um autómato celular. Neste problema vamos modificar as regras do autómato da seguinte forma:
- O espaço de estados é finito definido por uma grelha de células booleanas (morta=0/viva=1) de dimensão $\,N\times N\,$ (com $N>3$) identificadas por índices $\,(i,j)\in \{1..N\}$.  Estas $\;N^2\;$ células são aqui referidas como “normais”;
<br><br>
- Adicionalmente existem $\,2N+1\,$ “células da borda” que correspondem a um dos índices, $i$ ou $j$, ser zero. As células da borda têm valores constantes que, no estado inicial, são gerados aleatoriamente com uma probabilidade $\,\rho\,$ de estarem vivas;
<br><br>
- As células normais o autómato modificam o estado de acordo com a regra “B3/S23”: i.e. a célula nasce (passa de $0$ a $1$) se tem exatamente 3 vizinhos vivos e sobrevive (mantém-se viva) se o número de vizinhos vivos é 2 ou 3, caso contrário morre ou continua morta;

In [36]:
from pysmt.shortcuts import *
from pysmt.typing import BOOL, INT 
import random

### Inputs
São parâmetros do problema os parâmetros N, $\rho$ e a posição do centro.

In [73]:
N = 10 #dimensão da grelha (NxN) (N > 3)
ro = 0.5 #probabilidade ró de estarem vivas.
centro = (5,5) #posição do centro

        

In [74]:
quadrado = [(centro[0]+1, centro[1]), (centro[0], centro[1]+1),
           (centro[0]-1, centro[1]), (centro[0], centro[1]-1),
           (centro[0]+1, centro[1]+1), (centro[0]-1, centro[1]-1),
           (centro[0]+1, centro[1]-1), (centro[0]-1, centro[1]+1),
           (centro[0], centro[1])]


A função `declaracao` cria a i-ésima cópia das variáveis de estado, guardando-as num dicionário.

In [53]:
def declaracao(k, N):
    state = {}
    state['grelha'] = {}
    for i in range(N+1):
        for j in range(N+1):
            state['grelha'][i,j] = Symbol(f'{k}_grelha_{i}_{j}', INT)
            
    return state

A função `inicializacao` testa se um dado estado é um possível estado inicial do programa.

In [54]:
def inicializacao(state, N, quadrado, ro):
    expr = []
    
    for i in range(1, N+1):
        for j in range(1, N+1):
            if (i,j) not in quadrado:
                expr.append(Equals(state['grelha'][i,j], Int(0)))
            else:
                expr.append(Equals(state['grelha'][i,j], Int(1)))
           
    nums = [0,1]
    probs = [ro, 1-ro]
    expr.append(Equals(state['grelha'][0,0], Int(random.choices(nums, probs, k=1)[0])))
    i = 0
    for j in range(1, N+1):
        expr.append(Equals(state['grelha'][i,j], Int(random.choices(nums, probs, k=1)[0])))
        
    j = 0
    for i in range(1, N+1):
        expr.append(Equals(state['grelha'][i,j], Int(random.choices(nums, probs, k=1)[0])))
            

    return And(elem for elem in expr)

A função de `transicao` impõe as condições necessárias para que seja possível transitar para o estado seguinte

In [62]:
def transicao(curr, prox, N):
    expr = []
    
    for i in range(N+1):
        for j in range(N+1):
            vizinhos = []
            
            for x in range(-1,2):
                for y in range(-1,2):
                    
                    if x != 0 or y != 0:
                        if i+x >= 0 and i+x < N+1 and j+y >= 0 and j+y < N+1:
                            vizinhos.append(curr['grelha'][i+x,j+y])
                            
            soma_igual_3 = GE(Plus([elem for elem in vizinhos]), Int(3))
            celula_nasce = Equals(prox['grelha'][i,j], Int(1))
            soma_igual_2 = Equals(Plus([elem for elem in vizinhos]), Int(2))
            celula_mantem_se = Equals(prox['grelha'][i,j], curr['grelha'][i,j])
            celula_morre = Equals(prox['grelha'][i,j], Int(0))
            expr.append(Ite(soma_igual_3, celula_nasce, Ite(soma_igual_2, celula_mantem_se, celula_morre)))
    
    return And(expr)

In [75]:
def gera_traco(declaracao, inicializacao, transicao, k, N, quadrado):

    with Solver(name = "z3") as solver:
        # definir o traço de estados(arestas)
        traco = [declaracao(i, N) for i in range(k)]
        
        # adicionar o estado inicial
        solver.add_assertion(inicializacao(traco[0], N, quadrado, ro))
        
        # adicionar a função de transição
        for i in range(k-1):
            solver.add_assertion(transicao(traco[i], traco[i+1], N)) # condição ; # tem de ser verdadeira consoante o nosso estado atual e o seguinte
        
        
        if solver.solve():
            for i in range(k):
                print("Passo: ", i)
                for v in traco[i]:
                    for x in range(N+1):
                        for y in range(N+1):
                            print(solver.get_value(traco[i][v][x,y]), end = ' ')
                        print()
                print("-------------")

        else:
            print("Não encontrou solução")
        pass
    
    # completar
    
  
                
gera_traco(declaracao,inicializacao,transicao,10,N,quadrado)

Passo:  0
0 1 0 0 1 0 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 1 1 1 0 0 0 0 
0 0 0 0 1 1 1 0 0 0 0 
1 0 0 0 1 1 1 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  1
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
1 1 0 0 0 1 0 0 0 0 0 
0 0 0 0 1 1 1 0 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 
0 0 0 0 1 1 1 0 0 0 0 
1 1 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  2
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 1 1 0 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 
0 0 0 0 1 1 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  3
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 
0 0 1 1 1 1 1 1 1 0 0 
0 0 0 1 1 1 1 1 0 0 0

In [ ]:
def invariante_pelo_menos_uma_viva(state):
    